In [ ]:
!pip install gym[classic_control]
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ['SDL_VIDEODRIVER']='windib'
import pygame
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import random 
import math
import gym
from collections import deque

In [ ]:
episode = 100
start_eps = 0.9
end_eps = 0.05 
eps_decay = 200    
lr = 0.001
batch_size= 64
dis = 0.99

In [ ]:
class DQN:
    def __init__(self):
        self.model = nn.Sequential(
            nn.Linear(4, 256),
            nn.ReLU(),
            nn.Linear(256, 2)
        )
        self.optimizer = optim.Adam(self.model.parameters(), lr)
        self.steps = 0 
        self.memory = deque(maxlen=10000)

    def memorize(self, state, action, reward, next_state):
        self.memory.append((state,
                            action,
                            torch.FloatTensor([reward]),
                            torch.FloatTensor([next_state])))
    
    def act(self, state):
        eps_threshold = end_eps + (start_eps - end_eps) * math.exp(-1. * self.steps / eps_decay)
        self.steps += 1
        if random.random() > eps_threshold:
            return self.model(state).data.max(1)[1].view(1, 1)
        else:
            return torch.LongTensor([[random.randrange(2)]])
    
    def learn(self):
        if len(self.memory) < batch_size:
            return
        batch = random.sample(self.memory, batch_size)
        states, actions, rewards, next_states = zip(*batch)
        states = torch.cat(states)
        actions = torch.cat(actions)
        rewards = torch.cat(rewards)
        next_states = torch.cat(next_states)

        current_q = self.model(states).gather(1, actions)
        max_next_q = self.model(next_states).detach().max(1)[0]
        expected_q = rewards + (dis * max_next_q)

        loss = F.mse_loss(current_q.squeeze(), expected_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
env = gym.make('CartPole-v0')
DQN = DQN()
score = []

/usr/local/lib/python3.8/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
for e in range(1, episode+1):
    state = env.reset()
    steps = 0
    while True:
        env.render()
        state = torch.FloatTensor([state])
        action = DQN.act(state) 
        next_state, reward, done, _ = env.step(action.item())

        if done:
            reward = -1

        DQN.memorize(state, action, reward, next_state)
        DQN.learn()

        state = next_state
        steps += 1

        if done:
            print("횟수:{0} 점수: {1}".format(e, steps))
            score.append(steps)
            break

/usr/local/lib/python3.8/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


횟수:1 점수: 12
횟수:2 점수: 10
횟수:3 점수: 10
횟수:4 점수: 11
횟수:5 점수: 10
횟수:6 점수: 9
횟수:7 점수: 10
횟수:8 점수: 9
횟수:9 점수: 10
횟수:10 점수: 9
횟수:11 점수: 10
횟수:12 점수: 11
횟수:13 점수: 10
횟수:14 점수: 11
횟수:15 점수: 9
횟수:16 점수: 10
횟수:17 점수: 10
횟수:18 점수: 10
횟수:19 점수: 9
횟수:20 점수: 10
횟수:21 점수: 10
횟수:22 점수: 8
횟수:23 점수: 9
횟수:24 점수: 10
횟수:25 점수: 10
횟수:26 점수: 9
횟수:27 점수: 9
횟수:28 점수: 9
횟수:29 점수: 10
횟수:30 점수: 10
횟수:31 점수: 10
횟수:32 점수: 11
횟수:33 점수: 9
횟수:34 점수: 8
횟수:35 점수: 8
횟수:36 점수: 12
횟수:37 점수: 8
횟수:38 점수: 10
횟수:39 점수: 10
횟수:40 점수: 10
횟수:41 점수: 9
횟수:42 점수: 8
횟수:43 점수: 8
횟수:44 점수: 9
횟수:45 점수: 10
횟수:46 점수: 10
횟수:47 점수: 10
횟수:48 점수: 8
횟수:49 점수: 9
횟수:50 점수: 9
횟수:51 점수: 10
횟수:52 점수: 9
횟수:53 점수: 9
횟수:54 점수: 11
횟수:55 점수: 10
횟수:56 점수: 9
횟수:57 점수: 11
횟수:58 점수: 10
횟수:59 점수: 9
횟수:60 점수: 9
횟수:61 점수: 9
횟수:62 점수: 8
횟수:63 점수: 10
횟수:64 점수: 9
횟수:65 점수: 11
횟수:66 점수: 10
횟수:67 점수: 10
횟수:68 점수: 8
횟수:69 점수: 8
횟수:70 점수: 10
횟수:71 점수: 11
횟수:72 점수: 10
횟수:73 점수: 8
횟수:74 점수: 10
횟수:75 점수: 10
횟수:76 점수: 9
횟수:77 점수: 10
횟수:78 점수: 9
횟수:79 점수: 10
횟수:80 점수: 10
횟수: